## Create Dataframe with all towns and their Lat & Long

Dataset from
https://public.opendatasoft.com/explore/dataset/geonames-all-cities-with-a-population-1000/export/?disjunctive.cou_name_en&sort=name

API from https://www.breezometer.com/

In [16]:
####################IMPORT LIBRARIES####################
import pandas as pd
import requests
import re


####################INPUT PLACE NAME - City, Country####################
place = input('Where do you want to go? (City, Country)')

####################FIND REFERENCE FOR LAT & LONG COORDINATES TO FEED INTO API####################

#load dataset
geo = pd.read_csv('towns_latlong.csv', sep = ';')


#convert titles to lower case, change spaces for _
lower = []
under = []
for i in geo.columns:
    lower.append(i.lower())
for j in lower:
    under.append(j.replace(' ','_'))
geo.columns = under


#filter columns in dataframe
geo = geo[['geoname_id','name','alternate_names','country_name_en','country_code','timezone','label_en','coordinates' ]]


#split geocode to isolate both latitude and longitude
geo['latitude'] = geo['coordinates'].str.split(",", n=1, expand=True)[0]
geo['longitude'] = geo['coordinates'].str.split(",", n=1, expand=True)[1]


#split the searched location as two values: city and country
city = place.split(', ')[0]
country = place.split(', ')[1]


#filter df to match searched location, then isolate the latitude and longitude as string 
search = geo[(geo['name'] == city) & (geo['country_name_en'] == country)]
lat = search['latitude'].item()
long = search['longitude'].item()


####################EXTRACT API####################


#extract the api and change it into json format
breez = requests.get("https://api.breezometer.com/weather/v1/forecast/hourly?lat="+lat+"&lon="+long+"&key=bfbeb506ee514a02a07f4aa9a06e65da&hours=24")
print("Breezometer:",breez.status_code)

import json
bre = pd.DataFrame(breez.json())
bre['data'][0]


#extract the necessary information of times to define morning, afternoon and evening
counter = 0
ninem = []
three = []
ninen = []
for i in bre['data']:
    if '09:00:00' in bre['data'][counter]['datetime'].split('T')[1]:
        ninem.append(bre['data'][counter])
        counter += 1
    elif '15:00:00' in bre['data'][counter]['datetime'].split('T')[1]:
        three.append(bre['data'][counter])
        counter += 1
    elif '21:00:00' in bre['data'][counter]['datetime'].split('T')[1]:
        ninen.append(bre['data'][counter])
        counter += 1
    else:
        counter +=1

        
#Transform the information into dataframe to score the location's information at the specified times with geo info 
def weatherinfo(x):
    date = x[0]['datetime'].split('T')[0]
    time = x[0]['datetime'].split('T')[1].replace('Z','')
    #morntime = morntime.replace('Z','')
    felt_temp_c = round(x[0]['feels_like_temperature']['value'])
    precipitation_prob = round(ninem[0]['precipitation']['precipitation_probability'])
    windspeed_km_h = round(x[0]['wind']['speed']['value'],1)
    isday = ninem[0]['is_day_time']

    return pd.DataFrame({'location': [place]
                        ,'latitude':[lat]
                        ,'longitude':[long]
                        ,'date' : [date]
                        , 'time': [time]
                        , 'felt_temp_c' : [felt_temp_c]
                        , 'precipitation_prob' : [precipitation_prob]
                        , 'windspeed_km_h': [windspeed_km_h]
                        , 'day_time': [isday]})

morndf = weatherinfo(ninem)
morndf['time_of_day'] = 'morning'

aftdf = weatherinfo(three)
aftdf['time_of_day'] = 'afternoon'

evedf = weatherinfo(ninen)
evedf['time_of_day'] = 'evening'

weatherinfo = pd.concat([morndf,aftdf,evedf], axis = 0)
        



Where do you want to go? (City, Country)Glasgow, United Kingdom
Breezometer: 200


### Get information from city and output weather data for morning, afternoon, evening

In [17]:
weatherinfo

,location,latitude,longitude,date,time,felt_temp_c,precipitation_prob,windspeed_km_h,day_time,time_of_day
0,"Glasgow, United Kingdom",55.86515,-4.25763,2022-08-08,09:00:00,14,0,12.9,True,morning
0,"Glasgow, United Kingdom",55.86515,-4.25763,2022-08-07,15:00:00,16,0,19.2,True,afternoon
0,"Glasgow, United Kingdom",55.86515,-4.25763,2022-08-07,21:00:00,13,0,10.9,True,evening
